In [1]:
import cv2
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import pickle
from random import shuffle
from scipy.misc import imread
from scipy.misc import imresize
import tensorflow as tf

Using TensorFlow backend.


In [2]:
#dataset
import hickle as hkl
import numpy as np
from keras.preprocessing.image import Iterator

In [3]:
import os
def _load_image_set_idx(_data_root_path,_image_set):
    image_set_file = os.path.join(
        _data_root_path, 'ImageSets', _image_set+'.txt')
    assert os.path.exists(image_set_file), \
        'File does not exist: {}'.format(image_set_file)

    with open(image_set_file) as f:
      image_idx = [x.strip() for x in f.readlines()]
    return image_idx

In [4]:
idx_train = _load_image_set_idx('KITTI','train')
print(len(idx_train))

3740


In [5]:
idx_trainval = _load_image_set_idx('KITTI','trainval')
print(len(idx_trainval))

7481


In [6]:
idx_val = _load_image_set_idx('KITTI','val')
print(len(idx_val))

3741


In [10]:
def _get_idx_path(idx, img_path):
    image_path = os.path.join(img_path, idx+'.png')
    assert os.path.exists(image_path), \
        'Image does not exist: {}'.format(image_path)
    return image_path

In [12]:
print(_get_idx_path(idx_train[0], './KITTI/training/image_2/'))

./KITTI/training/image_2/000000.png


In [15]:
def _read_annotation(idx, annotation_path):
    def _get_obj_level(obj):
      height = float(obj[7]) - float(obj[5]) + 1
      trucation = float(obj[1])
      occlusion = float(obj[2])
      if height >= 40 and trucation <= 0.15 and occlusion <= 0:
          return 1
      elif height >= 25 and trucation <= 0.3 and occlusion <= 1:
          return 2
      elif height >= 25 and trucation <= 0.5 and occlusion <= 2:
          return 3
      else:
          return 4
    idx2annotation = {}
    for index in idx:
      filename = os.path.join(annotation_path, index+'.txt')
      with open(filename, 'r') as f:
        lines = f.readlines()
      f.close()
      bboxes = []
      for line in lines:
        obj = line.strip().split(' ')
        cls = obj[0].lower().strip()
        if _get_obj_level(obj) > 3:
          continue
        xmin = float(obj[4])
        ymin = float(obj[5])
        xmax = float(obj[6])
        ymax = float(obj[7])
        assert xmin >= 0.0 and xmin <= xmax, \
            'Invalid bounding box x-coord xmin {} or xmax {} at {}.txt' \
                .format(xmin, xmax, index)
        assert ymin >= 0.0 and ymin <= ymax, \
            'Invalid bounding box y-coord ymin {} or ymax {} at {}.txt' \
                .format(ymin, ymax, index)
        #x, y, w, h = bbox_transform_inv([xmin, ymin, xmax, ymax])
        #bboxes.append([x, y, w, h, cls])
        bboxes.append([xmin, ymin, xmax, ymax, cls])
      idx2annotation[index] = bboxes

    return idx2annotation

In [19]:
#saving to csv
df = _read_annotation(idx_train, './KITTI/training/label_2/')
print(df)

{'001521': [[411.26, 190.43, 455.68, 223.31, 'car'], [268.57, 193.58, 394.03, 281.18, 'car'], [470.75, 170.83, 531.2, 202.17, 'dontcare']], '000269': [[53.86, 191.17, 118.05, 244.56, 'car'], [272.75, 187.98, 337.13, 235.6, 'car'], [243.61, 188.82, 302.61, 230.57, 'car'], [63.32, 162.23, 126.03, 224.57, 'truck'], [223.23, 188.71, 257.16, 217.14, 'car'], [300.41, 186.75, 397.0, 250.73, 'car'], [160.14, 179.23, 214.01, 204.49, 'dontcare']], '003913': [[541.15, 186.73, 570.18, 216.14, 'car']], '002638': [[721.61, 191.36, 904.81, 306.64, 'car'], [34.69, 192.33, 330.64, 292.92, 'car'], [155.86, 186.89, 384.06, 271.18, 'car'], [261.2, 180.3, 426.9, 246.87, 'car'], [657.1, 183.94, 740.65, 240.25, 'car'], [361.78, 184.06, 497.46, 230.18, 'car'], [647.89, 181.72, 697.32, 215.36, 'car'], [378.39, 180.76, 484.6, 223.99, 'car'], [397.76, 173.41, 485.93, 217.18, 'car'], [632.25, 174.29, 662.0, 200.55, 'car'], [497.92, 169.53, 628.24, 204.11, 'dontcare']], '007099': [[589.69, 174.36, 649.9, 223.34, '

In [48]:
#name = {'index':[],'cls':[],'x1':[],'y1':[],'x2':[],'y2':[]}
data = pd.DataFrame(columns=['index','cls','x1','y1','x2','y2'])
#data = pd.DataFrame({'idx':[],'cls':[],'x1':[],'y1':[],'x2':[],'y2':[]})
lists = []
for d in df:
    for i in range(len(df[d])):
        #print(df[d][0][0])
        #lists.append([d,df[i][0][4],df[i][0][0],df[i]])
        data2 = {'index':d,'cls':df[d][i][4],'x1':df[d][i][0],'y1':df[d][i][1],'x2':df[d][i][2],'y2':df[d][i][3]}
        data = data.append(data2, ignore_index=True)
    #print(len(df[d]))
data.head()

,index,cls,x1,y1,x2,y2
0,001521,car,411.26,190.43,455.68,223.31
1,001521,car,268.57,193.58,394.03,281.18
2,001521,dontcare,470.75,170.83,531.20,202.17
3,000269,car,53.86,191.17,118.05,244.56
4,000269,car,272.75,187.98,337.13,235.60


In [49]:
data.to_csv('kitti.csv')